In [1]:
import sys, os, git, time

import MDAnalysis as mda
from MDAnalysis.analysis import distances

current_path = os.getcwd()
git_repo = git.Repo(current_path, search_parent_directories=True)
git_path = git_repo.git.rev_parse("--show-toplevel")
sys.path.append(git_path+"/python-codes/")

from MinimizeEnergy import MinimizeEnergy
from MonteCarlo import MonteCarlo
from MolecularDynamics import MolecularDynamics

import numpy as np

In [11]:
ti = time.time()

me = MinimizeEnergy(maximum_steps=10,
    number_atoms=[50],
    epsilon=[0.1],
    sigma = [3],
    atom_mass = [1],
    box_dimensions=[20, 20, 20],
    neighbor=10,
    thermo = 2,
    cut_off = 9,
    dump = 2)
me.perform_energy_minimization()

tf = time.time()

print("time =", np.round(tf-ti,2), "s")

step epot maxF
0 19205.223 116670.444
2 13675.363 82003.661
4 8660.751 50094.269
6 4687.250 25289.235
8 2070.116 9890.340
10 780.838 2760.295
time = 0.14 s


In [3]:
ti = time.time()

self = MonteCarlo(maximum_steps=5,
    desired_mu = -1,
    inserted_type= 0,
    number_atoms=[5],
    epsilon=[0.1],
    sigma = [3],
    atom_mass = [1],
    box_dimensions=[20, 20, 20],
    neighbor=10,
    thermo = 100,
    cut_off = 9,
    dump = 2)
self.run()

tf = time.time()

print("time =", np.round(tf-ti,2), "s")

print(self.number_atoms)

time = 0.01 s
[8]


In [14]:
ti = time.time()

self = MolecularDynamics(maximum_steps = 1000,
    tau_temp = 100,
    number_atoms = [50],
    epsilon = [0.1],
    sigma = [3],
    atom_mass = [1],
    box_dimensions=[20, 20, 20],
    neighbor = 10,
    thermo = 100,
    cut_off = 9,
    dump = 1,
    initial_positions = me.atoms_positions)
self.run()

tf = time.time()

print("time =", np.round(tf-ti,2), "s")

time = 6.08 s
